In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from mmwrapper.src.api import get_runner

In [ ]:
runner = get_runner(r"C:\Users\philmarq\source\repos\MMWrapperRepo\mmwrapper\src\configs\object_detection\object_detection.yaml")

In [ ]:
runner.train()

# Instance seg inference

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#import pydensecrf.densecrf as dcrf

from mmdet.apis import init_detector, inference_detector

import numpy as np
import cv2



config_file = r"C:\Users\philmarq\source\repos\MMWrapperRepo\firstapitest\20230725_162809\vis_data\config.py"
checkpoint_file = r"C:\Users\philmarq\source\repos\MMWrapperRepo\firstapitest\best_coco_bbox_mAP_epoch_16.pth"

device = 'cuda:0'

model = init_detector(config_file, checkpoint_file, device=device)

# Load image


colors = [(255,0,0), (125,0,0), (0,255,0), (50, 255,0), (0,25,255), (0, 125, 0)]
root = r"C:\HSA\HSA-Data\projects\d55090aa-a315-4ae3-8242-b50e13e355f6\dataset\val2017"
for file in os.listdir(root):
    img = os.path.join(root, file)
    result = inference_detector(model, img)
    
    

    color = (255,0,0)
    image = cv2.imread(img)
    #image[image>0] = 0
    masks = result.pred_instances.masks.cpu().numpy()
    scores = result.pred_instances.scores.cpu().numpy()
    labels = result.pred_instances.labels.cpu().numpy()
    bboxes = result.pred_instances.bboxes.cpu().numpy()
    thresh = 0.75
    
    for mask, score,label, bbox in zip(masks, scores,  labels, bboxes):
        if score > thresh:
            if label == 0:  # For label == 0, draw contours
                mask = mask.astype(np.uint8)
                contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                #contours = connect_closest_contours(contours, image, angle_tolerance = 5, fixed_threshold=50, pixel_threshold = 3)
                #color  = np.random.rand(3) * 255
                cv2.drawContours(image, contours, -1, color, -1)
            else:  # For other labels, draw bounding boxes
                color = colors[label]
                x1, y1, x2, y2 = bbox.astype(np.int32)
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 1)
    
    cv2.imwrite(os.path.join("outputall", file), image)